In [1]:
!pip install -q SentencePiece transformers

In [1]:
from transformers import TFT5EncoderModel, T5Tokenizer
import numpy as np
import re
import gc

In [2]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False )

In [3]:
model = TFT5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5EncoderModel: ['decoder.block.6.layer.1.EncDecAttention.q.weight', 'decoder.block.20.layer.1.layer_norm.weight', 'decoder.block.18.layer.2.layer_norm.weight', 'decoder.block.14.layer.1.EncDecAttention.o.weight', 'decoder.block.21.layer.0.SelfAttention.v.weight', 'decoder.block.22.layer.2.layer_norm.weight', 'decoder.block.20.layer.1.EncDecAttention.o.weight', 'decoder.block.17.layer.0.SelfAttention.q.weight', 'decoder.block.8.layer.1.EncDecAttention.k.weight', 'decoder.block.23.layer.0.layer_norm.weight', 'decoder.block.9.layer.1.EncDecAttention.q.weight', 'decoder.block.18.layer.0.SelfAttention.v.weight', 'decoder.block.22.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.11.layer.0.SelfAttention.k.weight', 'decoder.block.10.layer.0.SelfAttention.k.weight', 'decoder.block.4.layer.1.EncDecAttention.v.weight', 'decoder.block.5.layer.0.SelfAttention.v.wei

All the weights of TFT5EncoderModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5EncoderModel for predictions without further training.


In [158]:
import os
# path where you save the protein sequence file(.fas)
fasta_dir = 'D:\\dataset\\protein-RNA binding\\protein_fasta\\fasta\\test_nonbind'
protein_fasta = os.listdir(fasta_dir)
sequences_Example = []
sequences_name = []
embedding_dir = 'D:\\dataset\\protein-RNA binding\\protein_fasta\\embedding\\test_nonbind'
for i in range(len(protein_fasta)):
    f = open(fasta_dir + '\\' + protein_fasta[i], 'r')
    name = protein_fasta[i].strip('.fa')
    sequences_name.append(name)
    data = f.readlines()
    seq = data[1].strip()
    seq = " ".join(seq)
    sequences_Example.append(seq)
    sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]
    ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")
    input_ids = ids['input_ids']
    attention_mask = ids['attention_mask']
    embedding = model(input_ids)
    embedding = np.asarray(embedding.last_hidden_state)
    attention_mask = np.asarray(attention_mask)
    features = [] 
    for seq_num in range(len(embedding)):
        seq_len = (attention_mask[seq_num] == 1).sum()
        seq_emd = embedding[seq_num][:seq_len-1]
        features.append(seq_emd)
    for i in range(len(features)):
        np.save(embedding_dir + '\\' + sequences_name[i] + '.npy', features[i])
    f.close()
    gc.collect()